In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-09-28"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
5735,2024-09-28,México Lnbp,00:00,Soles,Panteras,1.65,2.15,169.5,1.85,1.85,-3.5,1.95,1.72,txXRPBft,0.606061,0.465116,0.540541,0.540541,0.071177,196.092,25.707725,0.131100,2.4,1.341641,0.559017,174.00,2.264,0.573132,0.253150,49.0,160.760,25.066396,0.155924,0.6,1.341641,2.236068,143.28,2.074,0.280500,0.135246,-40.0,100,72,1.74,1.99,165.306,179.512,0.186081,0.000000,0.088629,0.73,0.146,4.452055,0.000000,0.0,0.000000,-5.00,-1.000,-1.150000,0.000000,0.0,0.000000
5736,2024-09-28,Noruega Blno,11:00,Centrum Tigers,Tromso,1.09,5.61,156.5,1.86,1.80,-13.5,1.95,1.72,hEmNq5qH,0.917431,0.178253,0.537634,0.555556,0.095684,159.076,87.231684,0.548365,1.8,1.643168,0.912871,271.44,2.026,0.976796,0.482130,-3.0,365.346,261.888583,0.716823,0.6,1.341641,2.236068,354.96,5.232,4.411017,0.843084,-46.0,87,87,3.12,4.08,123.432,532.322,0.954066,0.023184,0.088629,1.38,0.276,0.326087,0.703908,0.8,0.096092,-1.98,-0.396,-11.641414,0.420173,0.5,0.079827
5737,2024-09-28,Países Baixos Wbl Feminina,14:30,TopKip Lions F,Triple Threat F,1.01,9.35,143.5,1.83,1.83,-18.5,1.98,1.70,AmPodwfn,0.990099,0.106952,0.546448,0.546448,0.097051,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,1.138469,0.000000,0.107603,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
5738,2024-09-28,Paraguai Lnb,20:30,Dep. San Jose,Sol De America,1.01,26.00,150.5,1.88,1.78,-47.5,2.01,1.68,8vyPeXZi,0.990099,0.038462,0.531915,0.561798,0.028561,100.552,18.066930,0.179677,2.4,1.341641,0.559017,129.36,1.254,0.296699,0.236602,85.0,696.352,768.627320,1.103791,0.6,1.341641,2.236068,192.00,10.408,10.306751,0.990272,-72.0,77,50,1.68,3.84,104.444,908.448,1.308449,0.038640,0.126474,0.13,0.026,0.384615,0.794460,0.9,0.105540,21.00,4.200,5.952381,0.000000,0.0,0.000000
5739,2024-09-28,Portugal Supercopa Feminina,12:00,Benfica F,Gdessa Barreiro F,1.26,3.35,128.5,1.83,1.83,-9.5,1.95,1.72,WdjkIcbL,0.793651,0.298507,0.546448,0.546448,0.092158,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.641151,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5840,2024-09-28,Eslovênia Liga Otp Banka,08:00,KK Triglav Kranj,Tajfun Sentjur,1.83,1.88,151.5,1.82,1.88,-2.5,2.05,1.65,zsgPaHYH,0.546448,0.531915,0.549451,0.531915,0.078363,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0,0,0.00,0.00,0.000,0.000,0.019059,0.022933,0.152888,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
5841,2024-09-28,Eslováquia Extraliga,13:00,Spisski Rytieri,Nitra Blue Wings,1.38,2.72,163.5,1.83,1.83,-7.5,2.05,1.65,IFNAMnwh,0.724638,0.367647,0.546448,0.546448,0.092285,236.930,149.288269,0.630094,1.2,1.643168,1.369306,116.48,3.090,2.

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
5751,13:00,Hungria Nacional,NKA Pecs,OSE Lions,1.68,Back Home
5763,15:00,França Lfb Feminina,Flammes Carolo F,Landes F,2.43,Back Home
5794,14:00,Áustria Superliga,Traiskirchen,Kapfenberg,1.70,Back Home
5800,14:30,Alemanha Pro A,Artland,Bayreuth,1.56,Back Home
